In [1]:
import model
import numpy as np
from keras.models import *
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import plot_model



# gray_data_model_1.hdf5: 保持Transpose以及1 输入gray图  数据集是gray_data_1000.npy batch 30 查看画图保存
# gray_data_model_2.hdf5: 保持Transpose以及1 输入gray图  数据集是gray_data_1000.npy batch 14 查看画图保存
# data_mask_2000_model_1.hdf5: 保持Transpose以及1 输入gray图  数据集是data_mask_2000_.npy batch 50 查看画图保存
# test.hdf5: 云端运行test 保持Transpose以及1 输入gray图  数据集是data_2000_.npy batch 30 查看画图保存
# test2.hdf5: 调整其filter为64（保持一致！） 保持Transpose以及1 输入gray图  数据集是data_2000_.npy batch 20 查看画图保存
# 效果并不是很好，并且需要调整dropout以及阈值分割0，1出来，这样更加明显
# TO DO
# 优化predict.py (可以从数据读取到结果输出不需要手动每次调整)
# 每个文件有的可以写成变量模式 全部写成变量(不要手动输入)
# 可视化 每层activation 以及filter

# problem
# 以前的模型filter 其中没有保持一致64 有的变成了96？（所以可以尝试rgb输入的情况）
class Main(object):
    def __init__(self, img_rows=256, img_cols=256):
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):
        train_image = np.load("data/data_2000_.npy")
        test_image = train_image
        train_image = train_image[0:1800].astype('float') / 255

        train_image_mask = np.load("data/data_mask_2000_.npy")
        test_image_mask = train_image_mask
        train_image_mask = train_image_mask[0:1800].astype('float') / 255

        test_image = test_image[1800:1810].astype('float') / 255

        test_image_mask = test_image_mask[1800:1810].astype('float') / 255

        return train_image, train_image_mask, test_image

    def train(self):
        print("-----------Loading data-------------")
        train_image, train_image_mask, test_image = self.load_data()
        print('-----------Loading data done--------')
        model1 = model.unet()
        # plot_model(model1, to_file='/home/xingyu/Desktop/model.png', show_shapes=True)
        model_checkpoint = ModelCheckpoint('model/test2.hdf5', monitor='loss', verbose=1, save_best_only=True)
        print('-----------Fitting Model------------')
        history = model1.fit(
            train_image,
            train_image_mask,
            batch_size=16,
            epochs=20,
            verbose=1,
            validation_split=0.2,
            shuffle=True,
            callbacks=[model_checkpoint]
        )
        print('-----------Plotting-----------------')
        acc = history.history['acc']
        val_acc = history.history['val_acc']
        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs = range(1, len(acc) + 1)
        plt.figure(num=1)
        plt.plot(epochs, loss, 'bo', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training & validation loss')
        plt.xlabel('epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig('plot/test2-loss.png')

        plt.figure(num=2)
        plt.plot(epochs, acc, 'bo', label='Training acc')
        plt.plot(epochs, val_acc, 'b', label='Validation acc')
        plt.title('Training & validation acc')
        plt.xlabel('epochs')
        plt.ylabel('acc')
        plt.legend()
        plt.savefig('plot/test2-acc.png')

        print('-----------Predict test data--------')
        image_mask_test = model1.predict(test_image, batch_size=16, verbose=1)
        np.save('results/test2.npy', image_mask_test)

    def save_img(self):
        print('-----------Array to Image--------')
        imgs = np.load('results/test2.npy')
        for i in range(imgs.shape[0]):
            img = imgs[i]
            img = image.array_to_img(img)
            img.save("results/test2-_%d.jpg" % (i))


if __name__ == '__main__':
    mynet = Main()
    mynet.train()
    mynet.save_img()


/home/dev_halolong/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


-----------Loading data-------------
-----------Loading data done--------
maxpooling1 output shape:  (?, 128, 128, 64)
maxpooling2 output shape:  (?, 64, 64, 64)
maxpooling3 output shape:  (?, 32, 32, 64)
maxpooling4 output shape:  (?, 16, 16, 64)
maxpooling5 output shape:  (?, 8, 8, 64)
upCon1 output shape:  (?, ?, ?, 64)
upCon2 output shape:  (?, ?, ?, 64)
upCon3 output shape:  (?, ?, ?, 64)
upCon4 output shape:  (?, ?, ?, 64)
upCon5 output shape:  (?, ?, ?, 64)
out1 shape: (?, 256, 256, 1)
out2 shape: (?, 256, 256, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0

Train on 1440 samples, validate on 360 samples
Epoch 1/20
1440/1440 [==============================] - 52s 36ms/step - loss: 0.2651 - acc: 0.9022 - val_loss: 0.2150 - val_acc: 0.9281

Epoch 00001: loss improved from inf to 0.26507, saving model to model/test2.hdf5
Epoch 2/20
1440/1440 [==============================] - 33s 23ms/step - loss: 0.1868 - acc: 0.9319 - val_loss: 0.1774 - val_acc: 0.9387

Epoch 00002: loss improved from 0.26507 to 0.18679, saving model to model/test2.hdf5
Epoch 3/20
1440/1440 [==============================] - 33s 23ms/step - loss: 0.1593 - acc: 0.9413 - val_loss: 0.1878 - val_acc: 0.9384

Epoch 00003: loss improved from 0.18679 to 0.15927, saving model to model/test2.hdf5
Epoch 4/20
1440/1440 [==============================] - 33s 23ms/step - loss: 0.1324 - acc: 0.9508 - val_loss: 0.1666 - val_acc: 0.9441

Epoch 00004: loss improved from 0.15927 to 0.13243, saving model to model/test2.hdf5
Epoch 5/20
1440/1440 [==============================] - 33s 23ms/step